In [4]:
pip install azure-storage-blob xlrd==1.2.0

Note: you may need to restart the kernel to use updated packages.


'C:\Users\Beatrice' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import translate, explode, split
import pandas as pd
import os

In [5]:
# start spark session
spark = SparkSession.builder.appName("app").getOrCreate()
spark.conf.set("fs.azure.account.key.cahospitalstorage.blob.core.windows.net", \
    "w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==")

In [6]:
# connect to blob storage and download excel file
container = "acceptedinsurance"
blob_location = "raw_data/HospitalAcceptedInsurances.csv"

blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=cahospitalstorage;AccountKey=w3EYeDH//L+J0uscsn3BYDatbFCcB7CUXK4MfOSNmwc16v3WAQ7EPcw2IpchUrelLVuowjfugmv99e0Os8J5Sw==;EndpointSuffix=core.windows.net")
container_client = blob_service_client.get_container_client(container)
blob_data = container_client.get_blob_client(blob_location)
with open("temp.txt", "wb") as temp:
  data = blob_data.download_blob()
  data.readinto(temp)

In [13]:
# read csv file
columns = ['Hosptial', 'AcceptedInsurances', 'Website']
schema = StructType([
    StructField('Hosptial', StringType(), False),
    StructField('AcceptedInsurances', StringType(), True),
    StructField('Website', StringType(), True)
])
pdf = pd.read_csv("temp.txt", skiprows=[0], names=columns)
df = spark.createDataFrame(pdf, schema=schema)

In [14]:
# Clean up Accepted Insurances column
df_array = df.withColumn('AcceptedInsurances', translate("AcceptedInsurances", "[]", ""))
df_final = df_array.withColumn('AcceptedInsurances', explode(split('AcceptedInsurances', ',')))

In [15]:
# Export df_final to blob storage
output = "clean_data_acceptedinsurance"
df_final.write.option("header", "true").mode("overwrite").parquet(output)
for file in os.listdir("/dbfs/" + output):
  try:
    with open("/dbfs/%s/%s" % (output, file), "rb") as data:
        container_client.upload_blob(name="%s/%s" % (output, file), data=data, overwrite=True)
  except Exception as e:
    print(e)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/dbfs/clean_data_acceptedinsurance'

In [0]:
dbutils.notebook.exit("%s/%s" % (container, output))